In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['KMP_WARNINGS'] = 'off'
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import re

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

In [2]:
lumi = lumiMap[2018][0]
mc = 5


In [3]:
n_bkg = 1000

bkg = Tree([], sample='bkg', color='lightgrey')
f_bkg_m = lambda n : np.sqrt( np.sum(np.random.exponential(100, size=(n, 2))**2, axis=1) )
f_bkg_pt = lambda n : np.random.exponential(100, size=(n,))

bkg.extend(
    m=f_bkg_m(mc*n_bkg),
    pt=f_bkg_pt(mc*n_bkg),
    scale=np.ones(mc*n_bkg)/(mc*lumi),
)

0it [00:00, ?it/s]

[WARNING] unable to open any files with filelist
           []


In [4]:
n_sig = 100

sig = Tree([], sample='sig', color='red', is_signal=True)
f_sig_m = lambda n : np.random.normal(125, 10, size=(n,))
f_sig_pt = lambda n : np.sqrt( np.sum(np.random.exponential(100, size=(n, 2))**2, axis=1) )
sig.extend(
    m=f_sig_m(mc*n_sig),
    pt=f_sig_pt(mc*n_sig),
    scale=np.ones(mc*n_sig)/(mc*lumi),
)

0it [00:00, ?it/s]

[WARNING] unable to open any files with filelist
           []


In [5]:
r_sig = np.random.uniform(0.8,1.2)
n_data = n_bkg + int(r_sig*n_sig)


data = Tree([], sample='data', color='black', is_data=True)
data.extend(
    m=np.concatenate([ f_bkg_m(n_bkg), f_sig_m(int(r_sig*n_sig)) ]),
    pt=np.concatenate([ f_bkg_pt(n_bkg), f_sig_pt(int(r_sig*n_sig)) ]),
    scale=np.ones(n_data),
)

0it [00:00, ?it/s]

[WARNING] unable to open any files with filelist
           []


In [8]:
np.array( np.random.uniform(size=(20,)) )

array([0.03845712, 0.06329529, 0.43057203, 0.7684302 , 0.11832377,
       0.85929631, 0.14767074, 0.48088462, 0.50723045, 0.39822332,
       0.19256811, 0.59228993, 0.51568119, 0.55712896, 0.71721395,
       0.05441485, 0.53814979, 0.07516243, 0.188933  , 0.81556641])

In [49]:
class Systematic:
    valid_types = ['lnN', 'shape']
    value_types = dict(
        lnN = float,
        shape = np.array,
    )

    def __init__(self, name, type, value):
        assert type in self.valid_types, f'Invalid systematic type {type}'
        
        self.name = name
        self.type = type
        self.value = self.value_types[type](value)

class Process:
    def __init__(self,
                 name,
                 counts,
                 errors,
                 bins,
                 category='bin1',
                 is_signal=False,
                 is_data=False,
                 ):
        assert not (is_signal and is_data), 'Process cannot be both signal and data'

        self.name = name
        self.category = category
        self.counts = counts
        self.errors = errors
        self.bins = bins
        self.is_signal = is_signal
        self.is_data = is_data

        self.systematics = []

    def add_systematic(self, name, type, value):
        self.systematics.append(Systematic(name, type, value))

    def add_systematics(self, systematics : list):
        for systematic in systematics:
            self.add_systematic(*systematic)


    

In [50]:
h_sig = Histo.from_tree(sig, 'm', bins=np.linspace(0, 300, 30), histtype='step')
h_bkg = Histo.from_tree(bkg, 'm', bins=np.linspace(0, 300, 30))
h_data = Histo.from_tree(data, 'm', bins=np.linspace(0, 300, 30), marker='o')

In [51]:
p_sig = Process(h_sig.label, h_sig.histo, h_sig.error, h_sig.bins, is_signal=True)

p_sig.add_systematics([
    ['lumi' ,'lnN', 1.023 ],
    ['bTag' ,'lnN', 1.086   ],
    ['JER' ,'lnN', 1.021   ],
    ['JEC' ,'lnN', 1.029   ],
    ['trigger' ,'lnN', 1.090 ] ,
    ['PDF' ,'lnN', 1.035   ],
    ['xS' ,'lnN', 1.050 ],
])

p_bkg = Process(h_bkg.label, h_bkg.histo, h_bkg.error, h_bkg.bins)

p_bkg.add_systematics([
    ['xB', 'lnN', 1.050],
])

p_data = Process(h_data.label, h_data.histo, h_data.error, h_data.bins, is_data=True)

In [52]:
class Workspace:

    def __init__(self):
        self.signal_processes = []
        self.background_processes = []
        self.data = None

    def add_signal_process(self, process):
        self.signal_processes.append(process)

    def add_background_process(self, process):
        self.background_processes.append(process)

    def set_data(self, process):
        self.data = process

    @property
    def processes(self): return self.signal_processes + self.background_processes

In [53]:
ws = Workspace()

ws.add_signal_process(p_sig)
ws.add_background_process(p_bkg)
ws.set_data(p_data)

In [54]:
datacard = """
imax * number of channels
jmax * number of backgrounds
kmax * number of nuisance parameters (sources of systematical uncertainties)

shapes * * {filename} $PROCESS $PROCESS_$SYSTEMATIC
----
bin bin1
observation -1
bin           {categories}  
process		  {process_names}  
process		  {process_ids}
rate		  {process_rates}
----
{systematics}
* autoMCStats 0
"""

In [55]:
dict(
    categories=list(map(lambda f : f.category, ws.processes)),
    process_names=list(map(lambda f : f.name, ws.processes)),
    process_ids=(-np.arange( len(ws.signal_processes) ) ).tolist() + (np.arange( len(ws.background_processes) ) + 1).tolist(),
    process_rates = [-1]*len(ws.processes),
)

{'categories': ['bin1', 'bin1'],
 'process_names': ['sig', 'bkg'],
 'process_ids': [0, 1],
 'process_rates': [-1, -1]}

In [56]:
systematics = defaultdict(lambda : [None]*len(ws.processes))

for i, process in enumerate(ws.processes):
    for systematic in process.systematics:
        systematics[systematic.name][i] = 

lumi lnN 1.023
lumi lnN 1.023
bTag lnN 1.086
JER lnN 1.021
JEC lnN 1.029
trigger lnN 1.09
PDF lnN 1.035
xS lnN 1.05
xB lnN 1.05
